In [30]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import sympy as sym

In [31]:
q = 3e-4
L = 5 
g = 10
w = 114.6
k = 9e9

constante = (((k**2)*(q**4))/((L**4)*(w**2)))*((((4*np.sqrt(2))+2)/8)**2)

In [32]:
constante

0.07322199739508199

In [33]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [34]:
def f(x,y,c):
    
    z = x + sym.I*y
    f = sym.sin(z)**6 + (c*(sym.sin(z)**2))-c #- dependen si me piden con polinomio o con senos (z)**6 + (c*(z)**2) - c 
    f = f.expand()
    return sym.re(f),sym.im(f)

In [35]:
f0,f1 = f(x,y,constante)
F = [f0,f1]
F

[sin(x)**6*cosh(y)**6 - 15*sin(x)**4*cos(x)**2*sinh(y)**2*cosh(y)**4 + 15*sin(x)**2*cos(x)**4*sinh(y)**4*cosh(y)**2 + 0.073221997395082*sin(x)**2*cosh(y)**2 - cos(x)**6*sinh(y)**6 - 0.073221997395082*cos(x)**2*sinh(y)**2 - 0.073221997395082,
 6*sin(x)**5*cos(x)*sinh(y)*cosh(y)**5 - 20*sin(x)**3*cos(x)**3*sinh(y)**3*cosh(y)**3 + 6*sin(x)*cos(x)**5*sinh(y)**5*cosh(y) + 0.146443994790164*sin(x)*cos(x)*sinh(y)*cosh(y)]

In [36]:
J = sym.zeros(2,2)

for i in range(2):
    for j in range(2):
        if j==0:
            J[i,j] = sym.diff(F[i],x,1)
        else:
            J[i,j] = sym.diff(F[i],y,1)

In [37]:
J

Matrix([
[     30*sin(x)**5*cos(x)*sinh(y)**2*cosh(y)**4 + 6*sin(x)**5*cos(x)*cosh(y)**6 - 60*sin(x)**3*cos(x)**3*sinh(y)**4*cosh(y)**2 - 60*sin(x)**3*cos(x)**3*sinh(y)**2*cosh(y)**4 + 6*sin(x)*cos(x)**5*sinh(y)**6 + 30*sin(x)*cos(x)**5*sinh(y)**4*cosh(y)**2 + 0.146443994790164*sin(x)*cos(x)*sinh(y)**2 + 0.146443994790164*sin(x)*cos(x)*cosh(y)**2, 6*sin(x)**6*sinh(y)*cosh(y)**5 - 60*sin(x)**4*cos(x)**2*sinh(y)**3*cosh(y)**3 - 30*sin(x)**4*cos(x)**2*sinh(y)*cosh(y)**5 + 30*sin(x)**2*cos(x)**4*sinh(y)**5*cosh(y) + 60*sin(x)**2*cos(x)**4*sinh(y)**3*cosh(y)**3 + 0.146443994790164*sin(x)**2*sinh(y)*cosh(y) - 6*cos(x)**6*sinh(y)**5*cosh(y) - 0.146443994790164*cos(x)**2*sinh(y)*cosh(y)],
[-6*sin(x)**6*sinh(y)*cosh(y)**5 + 60*sin(x)**4*cos(x)**2*sinh(y)**3*cosh(y)**3 + 30*sin(x)**4*cos(x)**2*sinh(y)*cosh(y)**5 - 30*sin(x)**2*cos(x)**4*sinh(y)**5*cosh(y) - 60*sin(x)**2*cos(x)**4*sinh(y)**3*cosh(y)**3 - 0.146443994790164*sin(x)**2*sinh(y)*cosh(y) + 6*cos(x)**6*sinh(y)**5*cosh(y) + 0.146443994790

In [38]:
InvJ = J.inv(method='LU')
InvJ

Matrix([
[((-6*sin(x)**6*sinh(y)*cosh(y)**5 + 60*sin(x)**4*cos(x)**2*sinh(y)**3*cosh(y)**3 + 30*sin(x)**4*cos(x)**2*sinh(y)*cosh(y)**5 - 30*sin(x)**2*cos(x)**4*sinh(y)**5*cosh(y) - 60*sin(x)**2*cos(x)**4*sinh(y)**3*cosh(y)**3 - 0.146443994790164*sin(x)**2*sinh(y)*cosh(y) + 6*cos(x)**6*sinh(y)**5*cosh(y) + 0.146443994790164*cos(x)**2*sinh(y)*cosh(y))*(6*sin(x)**6*sinh(y)*cosh(y)**5 - 60*sin(x)**4*cos(x)**2*sinh(y)**3*cosh(y)**3 - 30*sin(x)**4*cos(x)**2*sinh(y)*cosh(y)**5 + 30*sin(x)**2*cos(x)**4*sinh(y)**5*cosh(y) + 60*sin(x)**2*cos(x)**4*sinh(y)**3*cosh(y)**3 + 0.146443994790164*sin(x)**2*sinh(y)*cosh(y) - 6*cos(x)**6*sinh(y)**5*cosh(y) - 0.146443994790164*cos(x)**2*sinh(y)*cosh(y))/((30*sin(x)**5*cos(x)*sinh(y)**2*cosh(y)**4 + 6*sin(x)**5*cos(x)*cosh(y)**6 - 60*sin(x)**3*cos(x)**3*sinh(y)**4*cosh(y)**2 - 60*sin(x)**3*cos(x)**3*sinh(y)**2*cosh(y)**4 + 6*sin(x)*cos(x)**5*sinh(y)**6 + 30*sin(x)*cos(x)**5*sinh(y)**4*cosh(y)**2 + 0.146443994790164*sin(x)*cos(x)*sinh(y)**2 + 0.1464439947901

In [39]:
Fn = sym.lambdify([x,y],F,'numpy')
IJn = sym.lambdify([x,y],InvJ,'numpy')

In [40]:
def NewtonRaphson(z,Fn,Jn,itmax=100,precision=1e-20):
    
    error = 1
    it = 0
    
    while error > precision and it < itmax:
        
        IFn = Fn(z[0],z[1])
        IJn = Jn(z[0],z[1])
        
        z1 = z - np.dot(IJn,IFn)
        
        error = np.max( np.abs(z1-z) )
        
        z = z1
        it +=1
        
    #print(it)
    return z

In [41]:

x_range = np.linspace(-3.5, 3.5, 10)
y_range = np.linspace(-3, 3, 8)

roots = []
roots_reals = []
for x_val in x_range:
    for y_val in y_range:
        z0 = np.array([x_val, y_val])
        zsol = NewtonRaphson(z0, Fn, IJn)
        zsol =  np.round(zsol,5)
        roots.append(zsol)

unique_roots = [tuple(root) for root in np.unique(roots, axis=0)]



for roots in unique_roots:
    
    if roots[1]==0.0 or roots[1]==-0.0:
        roots_reals.append(roots[0])
      
        
        

    
    
   




[(-3.46358, -0.56203), (-3.46358, 0.56203), (-2.81961, -0.56203), (-2.81961, 0.56203), (-2.49763, 0.0), (-0.64397, -0.0), (-0.32198, -0.56203), (-0.32198, 0.56203), (0.32198, -0.56203), (0.32198, 0.56203), (0.64397, -0.0), (2.49763, -0.0), (2.81961, -0.56203), (2.81961, 0.56203), (3.46358, -0.56203), (3.46358, 0.56203)]


In [44]:
#Si me piden con z, utilizo este código:

# for teta in roots_reals:
#     angulo=np.arcsin(teta)
#     print(angulo*(180/np.pi))



nan
-40.08849346775175
40.08849346775175
nan


C:\Users\gaboe\AppData\Local\Temp\ipykernel_8060\1827268487.py:4: RuntimeWarning: invalid value encountered in arcsin
  angulo=np.arcsin(teta)


In [45]:
# Si me piden con senos, utilizo este 
        
for angulo_rad in roots_reals:
    angulo_grad=angulo_rad*(180/np.pi)
    print(angulo_grad)


-143.1036577852598
-36.89676313303963
36.89676313303963
143.1036577852598
